In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [43]:
import pickle

In [5]:
base_folder = "/content/drive/MyDrive/Projects/Resume_Matching"
jd_selected_data = pd.read_csv(base_folder + "/job_descriptions_selected_cleaned.csv")
resumes_all_data = pd.read_csv(base_folder + "/Resumes_Extracted_Sectionized_False.csv")

In [44]:
with open(base_folder + "/indices.pickle", 'rb') as f:
  jds_mapping = pickle.load(f)
jds_mapping

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 29, 30, 31, 32, 33, 34, 35, 36]

In [45]:
len(jds_mapping)

21

In [10]:
jd_vectors = torch.load(base_folder + "/JDs_all_desc_vectors.pt")
resumes_all_vectors = torch.load(base_folder + "/Resumes_all_vectors.pt")

In [11]:
jd_vectors = jd_vectors.detach().numpy()
resumes_all_vectors = resumes_all_vectors.detach().numpy()

In [12]:
jd_vectors.shape

(21, 768)

In [13]:
resumes_all_vectors.shape

(238, 768)

In [38]:
# KNN
selected_cvs = []
for i in range(len(jd_vectors)):
  sims = cosine_similarity([jd_vectors[i]], resumes_all_vectors)

  sorted_indices = np.argsort(sims[0])
  top_5_indices = sorted_indices[-5:]

  cvs = [(resumes_all_data["cv_category"][i], resumes_all_data["cv_number"][i]) for i in top_5_indices]
  selected_cvs.append(cvs)

In [39]:
len(selected_cvs)

21

In [40]:
scv = selected_cvs[0]

In [41]:
print(len(scv[0]))

2


In [42]:
print(selected_cvs[0])

[('ENGINEERING', 33685075), ('INFORMATION-TECHNOLOGY', 28126340), ('ENGINEERING', 56691064), ('ENGINEERING', 28630325), ('ENGINEERING', 14049846)]


In [46]:
content = {"JD_Index": [], "1": [], "2": [], "3": [], "4": [], "5": []}

In [47]:
content["JD_Index"] = jds_mapping

In [49]:
for i in range(len(jds_mapping)):
  selected_cvs_list = selected_cvs[i]
  for j in range(len(selected_cvs_list)):
    cv_type = selected_cvs_list[j][0]
    cv_number = selected_cvs_list[j][1]
    cv_name = '-'.join([cv_type, str(cv_number)])
    content[str(j+1)].append(cv_name)

df = pd.DataFrame(content)

In [50]:
df.head()

,JD_Index,1,2,3,4,5
0,3,ENGINEERING-33685075,INFORMATION-TECHNOLOGY-28126340,ENGINEERING-56691064,ENGINEERING-28630325,ENGINEERING-14049846
1,4,INFORMATION-TECHNOLOGY-18176523,ENGINEERING-82246962,ENGINEERING-36149549,INFORMATION-TECHNOLOGY-31111279,INFORMATION-TECHNOLOGY-46260230
2,5,ENGINEERING-38535335,INFORMATION-TECHNOLOGY-24038620,INFORMATION-TECHNOLOGY-24020470,INFORMATION-TECHNOLOGY-46260230,ENGINEERING-36149549
3,6,INFORMATION-TECHNOLOGY-20674668,INFORMATION-TECHNOLOGY-64017585,ENGINEERING-44624796,ENGINEERING-23497307,ENGINEERING-32081266
4,7,INFORMATION-TECHNOLOGY-15802627,INFORMATION-TECHNOLOGY-24020470,INFORMATION-TECHNOLOGY-31111279,ENGINEERING-36149549,INFORMATION-TECHNOLOGY-46260230


In [51]:
df.to_csv(base_folder + "/top_5_cvs_selected_jds.csv")